[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_pairs_trading/EMQA_pairs_trading.ipynb)

# EMQA_pairs_trading
Brent-WTI pairs trading strategy using z-score signals.
**Output:** `pairs_trading.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
# Fetch Brent and WTI
brent = fetch('BZ=F')
wti = fetch('CL=F')

common = brent.index.intersection(wti.index)
brent = brent.loc[common]
wti = wti.loc[common]

# Spread and z-score
spread = brent - wti
roll_mean = spread.rolling(30).mean()
roll_std = spread.rolling(30).std()
zscore = ((spread - roll_mean) / roll_std).dropna()

print(f"Spread: mean={spread.mean():.2f}, std={spread.std():.2f}")
print(f"Z-score range: [{zscore.min():.2f}, {zscore.max():.2f}]")

In [ ]:
# Generate trading signals
entry_threshold = 2.0
exit_threshold = 0.0

position = pd.Series(0.0, index=zscore.index)
for i in range(1, len(zscore)):
    if zscore.iloc[i] > entry_threshold:
        position.iloc[i] = -1  # short spread
    elif zscore.iloc[i] < -entry_threshold:
        position.iloc[i] = 1   # long spread
    elif abs(zscore.iloc[i]) < exit_threshold + 0.5:
        position.iloc[i] = 0   # exit
    else:
        position.iloc[i] = position.iloc[i-1]

# P&L: position * change in spread
spread_aligned = spread.loc[zscore.index]
daily_pnl = position.shift(1) * spread_aligned.diff()
daily_pnl = daily_pnl.fillna(0)
cum_pnl = daily_pnl.cumsum()

print(f"Total P&L: ${cum_pnl.iloc[-1]:.2f}")
print(f"Sharpe: {daily_pnl.mean() / daily_pnl.std() * np.sqrt(252):.2f}")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

# Panel 1 - Z-score with thresholds
axes[0].plot(zscore.index, zscore.values, color=COLORS['blue'], linewidth=0.8, label='Z-score')
axes[0].axhline(entry_threshold, color=COLORS['red'], linestyle='--', linewidth=1, label=f'Entry (+/-{entry_threshold})')
axes[0].axhline(-entry_threshold, color=COLORS['red'], linestyle='--', linewidth=1)
axes[0].axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)
axes[0].fill_between(zscore.index, -entry_threshold, entry_threshold, alpha=0.05, color=COLORS['green'])
axes[0].set_ylabel('Z-score')
axes[0].set_title('Brent-WTI Pairs Trading: Z-score Signal')

# Mark long/short signals
long_sig = (position == 1) & (position.shift(1) != 1)
short_sig = (position == -1) & (position.shift(1) != -1)
axes[0].scatter(zscore.index[long_sig], zscore[long_sig], color=COLORS['green'],
                marker='^', s=40, zorder=5, label='Long signal')
axes[0].scatter(zscore.index[short_sig], zscore[short_sig], color=COLORS['red'],
                marker='v', s=40, zorder=5, label='Short signal')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=5)

# Panel 2 - Position
axes[1].fill_between(position.index, position.values, 0, where=position > 0,
                      color=COLORS['green'], alpha=0.4, label='Long')
axes[1].fill_between(position.index, position.values, 0, where=position < 0,
                      color=COLORS['red'], alpha=0.4, label='Short')
axes[1].set_ylabel('Position')
axes[1].set_title('Trading Position')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), frameon=False, ncol=2)

# Panel 3 - Cumulative P&L
axes[2].plot(cum_pnl.index, cum_pnl.values, color=COLORS['blue'], linewidth=1.5)
axes[2].fill_between(cum_pnl.index, 0, cum_pnl.values,
                      where=cum_pnl >= 0, alpha=0.2, color=COLORS['green'])
axes[2].fill_between(cum_pnl.index, 0, cum_pnl.values,
                      where=cum_pnl < 0, alpha=0.2, color=COLORS['red'])
axes[2].axhline(0, color=COLORS['gray'], linestyle=':', linewidth=0.8)
axes[2].set_ylabel('Cumulative P&L ($)')
axes[2].set_xlabel('Date')
axes[2].set_title('Cumulative Profit & Loss')

plt.tight_layout()
save_fig(fig, 'pairs_trading.pdf')
plt.show()